In [2]:
import pandas as pd
import numpy as np
import scipy.stats

gams_python_path="/Library/Frameworks/GAMS.framwork/Resources/apifiles/Python/gams"

#### Import Data

In [50]:
model=pd.read_csv('/Users/hannahkamen/Downloads/windc_ind.csv')
cps=pd.read_stata('/Users/hannahkamen/Downloads/acs_20142017_5yr.dta')

lookup=pd.read_excel('/Users/hannahkamen/Downloads/windc_cps_industry_lookup.xlsx')
lookup_final=pd.read_excel('/Users/hannahkamen/Downloads/dropped_merge.xlsx')
state_lookup=pd.read_excel('/Users/hannahkamen/Downloads/state_lookup.xlsx')

lookup['NAICS']=lookup['NAICS'].astype(str).str.strip()
lookup_final['NAICS']=lookup_final['NAICS'].astype(str).str.strip()
lookup_final['cps_code']=lookup_final['cps_code'].astype(str).str.strip()

#cps = np.loadtxt('/Users/hannahkamen/Downloads/usa_00002.dat', unpack = True)



In [51]:
len(cps)

62630465

In [114]:
####gams labor demand data
gams_dta_ld0=pd.read_csv('/Users/hannahkamen/Downloads/windc_ld0_d2.csv')

####gams labor endowment data
gams_dta_le0=pd.read_csv('/Users/hannahkamen/Downloads/windc_le0_d.csv')


In [53]:
state_lookup['state']=state_lookup['state'].str.strip()
state_lookup['abbrev']=state_lookup['abbrev'].str.strip()

In [54]:
###read in dropped 

In [55]:
cps_codes=pd.DataFrame(data={'cps_code':cps['indnaics'].str.strip().unique()})
windc_codes=pd.DataFrame(data={'windc_code':model['IOCode'].str.strip().unique()})

In [56]:
####match on first three symbols
cps_codes['cps_code3']=cps_codes['cps_code'].str[0:3]
lookup['NAICS3']=lookup['NAICS'].str[0:3]

merge3=lookup.merge(cps_codes, left_on='NAICS3',right_on='cps_code3',how='inner',indicator=True)

In [57]:
###append manual lookup to fill gaps
merge_f=merge3[['NAICS','cps_code']].append(lookup_final)

In [80]:
###get state symbol for place of residence
cps['statefip']=cps['statefip'].str.lower().str.strip()
cps['pwstate2']=cps['pwstate2'].str.lower().str.strip()

cps_m=cps.merge(merge_f, left_on='indnaics', right_on='cps_code',how='left').merge(state_lookup, left_on='statefip',right_on='state', how='left')
cps_m=cps_m.rename(columns={'abbrev':'state_residence'})

###get state symbol for place of work
cps_m=cps_m.merge(state_lookup, left_on='pwstate2', right_on='state',how='left')
cps_m=cps_m.rename(columns={'abbrev':'state_work'})

### get skilled unskilled breakouts
cps_m['educ_new']=np.where(cps_m['educ'].isin(['2 years of college','4 years of college','5+ years of college']),'skl','unskl')
 


In [81]:
###prep household variable
cps_m['hh']=np.where(cps_m['inctot']<=25000,'hh1','')
cps_m['hh']=np.where(((cps_m['inctot']>25000) & (cps_m['inctot']<50000)),'hh2',cps_m['hh'])
cps_m['hh']=np.where(((cps_m['inctot']>=50000) & (cps_m['inctot']<75000)),'hh3',cps_m['hh'])
cps_m['hh']=np.where(((cps_m['inctot']>=75000) & (cps_m['inctot']<150000)),'hh4',cps_m['hh'])
cps_m['hh']=np.where((cps_m['inctot']>=150000),'hh5',cps_m['hh'])

In [96]:
non_s=['Output',
       'imports', 'ciffob', 'BasicSupply', 'Margins', 'TrnCost', 'TrdTrn',
       'Duties', 'Tax', 'Subsidies', 'TaxLesSubsidies', 'Supply']

states=cps_m['state_residence'].unique()

In [97]:
cps_m1=cps_m[(cps_m['state_work'].isin(states))& (cps_m['state_residence'].isin(states))]


In [99]:
test=cps_m[~(cps_m['state_work'].isin(states))]

In [100]:
test['state_work'].unique()

array([nan, 'PR'], dtype=object)

In [ ]:
#(cps_m['indnaics']!='0') & (cps_m['indnaics']!='') & 

In [98]:
len(cps_m1)

36212676

### Assign Sector Values and Limit Data

In [101]:
#list of US states

#### drop if unemployed
#cps_m1=cps_m[(cps_m['indnaics']!='0') & (cps_m['indnaics']!='') & (cps_m['state_work'].isin(states))]

#### group data by all relevant breakous - r,q,h,s,sk
all_breakouts0=cps_m1.reset_index()[['state_residence','state_work','NAICS','hh','educ_new','incwage']].groupby(['state_residence','state_work','NAICS','hh','educ_new'],as_index=False).agg({'incwage':sum})

#rename variables and limit dataframe to relevant sectors
all_breakouts0=all_breakouts0.rename(columns={'state_residence':'r','state_work':'q','NAICS':'s','hh':'h','educ_new':'sk'})
all_breakouts0=all_breakouts0[['r','q','s','h','sk','incwage']]

### now group model dataset by relevant Naics codes and Windc specific codes
model_gr=model.groupby(['IOCode','gams.IOCode'],as_index=False).sum()
model_gr=model_gr[['IOCode','gams.IOCode']]

###merge with cps data 
all_breakouts0=all_breakouts0.merge(model_gr, left_on='s',right_on='IOCode',how='left')

##get rid of used sector

all_breakouts0=all_breakouts0[all_breakouts0['gams.IOCode']!='use']

##now delete old NAICS var and re-assign ind var as the values in 'gams.IOCode'
del all_breakouts0['s']
del all_breakouts0['IOCode']
all_breakouts0=all_breakouts0.rename(columns={'gams.IOCode':'s'})
all_breakouts=all_breakouts0[['r','q','s','h','sk','incwage']]



In [102]:
len(all_breakouts)

147107

#### get current model breakouts region lived in,region worked, household type le0(r,q,h)

In [125]:
###classify census regions
midwest=['IA','OH','WI','NE','IL','MI','SD','ND','MN','MO','IN','KS']
south=['FL','MD','TN','WV','OK','KY','NC','VA','DE','GA','MS','TX','AL','LA','AR','SC','DC']
west=['AK','AZ','NM','HI','CA','WA','NV','OR','ID','UT','MT','WY','CO']
northeast=['VT','NH','CT','ME','MA','NY','NJ','PA','RI']

###get census region lived in, census region worked in

all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(midwest),'midwest','')
all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(south),'south',all_breakouts['census_region_r'])
all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(west),'west',all_breakouts['census_region_r'])
all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(northeast),'northeast',all_breakouts['census_region_r'])


all_breakouts['census_region_q']=np.where(all_breakouts['q'].isin(midwest),'midwest','')
all_breakouts['census_region_q']=np.where(all_breakouts['q'].isin(south),'south',all_breakouts['census_region_q'])
all_breakouts['census_region_q']=np.where(all_breakouts['q'].isin(west),'west',all_breakouts['census_region_q'])
all_breakouts['census_region_q']=np.where(all_breakouts['q'].isin(northeast),'northeast',all_breakouts['census_region_q'])

####limit to where person works in same census region that they live in
all_breakouts_lm=all_breakouts[(all_breakouts['census_region_r']==all_breakouts['census_region_q'])]

###get total income by r,q then get income by r,q,h calculate to get share household income by income type
#total by rqh
all_breakouts_lm_gr=all_breakouts_lm.groupby(['r','q','h'],as_index=False).agg({'incwage':sum})

##total by r,q
all_breakouts_lm_tot=all_breakouts_lm.groupby(['r','q'],as_index=False).agg({'incwage':sum})

all_breakouts_lm_tot=all_breakouts_lm_tot.rename(columns={'incwage':'incwage_total'})

le0_breakouts=all_breakouts_lm_gr.merge(all_breakouts_lm_tot, on=['r','q'],how='left')

le0_breakouts['hh_shr']=le0_breakouts['incwage']/le0_breakouts['incwage_total']


###get total income by r,s then get income by r,q,h calculate to get share household income by income type
#total by rqh
all_breakouts_lm_gr1=all_breakouts_lm.groupby(['r','s'],as_index=False).agg({'incwage':sum})

##total by r,q
all_breakouts_lm_tot1=all_breakouts_lm.groupby(['r'],as_index=False).agg({'incwage':sum})

all_breakouts_lm_tot1=all_breakouts_lm_tot1.rename(columns={'incwage':'incwage_total'})

ld0_breakouts=all_breakouts_lm_gr1.merge(all_breakouts_lm_tot1, on=['r'],how='left')

ld0_breakouts['hh_shr']=ld0_breakouts['incwage']/ld0_breakouts['incwage_total']



/var/folders/w1/12ls607j77q1jk587t2n5t2r0000gn/T/ipykernel_24676/1150673145.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(midwest),'midwest','')
/var/folders/w1/12ls607j77q1jk587t2n5t2r0000gn/T/ipykernel_24676/1150673145.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_breakouts['census_region_r']=np.where(all_breakouts['r'].isin(south),'south',all_breakouts['census_region_r'])
/var/folders/w1/12ls607j77q1jk587t2n5t2r0000gn/T/ip

#### Get all region lived in, region worked in combinations by sector

In [20]:
###group labor endowment by place of work and sector. These are all the place of work and sector combinations
### in the model that we need shares for
ld0=gams_dta_ld0.rename(columns={' flx':'q',' int':'s'})[['q','s']].groupby(['q','s'],as_index=False).sum()

##prepare le0 table
le00=gams_dta_le0.rename(columns={' flx':'r',' int':'q',' scn':'h',' esubw_oth':'sk',' sector':'pop'})[['r','q','h','sk','pop']]
le0=le00.groupby(['r','q','sk'],as_index=False).sum()
## merge region lived in and region worked in combinations with all sector combinations for the region worked in (q)
all_combs=le0.merge(ld0, on=['q'],how='inner')



In [1]:
le0_breakouts.head()

NameError: name 'le0_breakouts' is not defined

In [121]:
test2=test[~(test['NAICS'].isnull())]

In [128]:
gams_dta_ld0

,file,flx,int,scn,esubw_oth,sector,seniority,short,value
0,sm_test,IA,ppd,skl,0.226799,NaN,NaN,NaN,NaN
1,sm_test,IA,ppd,unskl,0.099038,NaN,NaN,NaN,NaN
2,sm_test,IA,res,skl,1.309129,NaN,NaN,NaN,NaN
3,sm_test,IA,res,unskl,0.571665,NaN,NaN,NaN,NaN
4,sm_test,IA,com,skl,0.779061,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7197,sm_test,CO,otr,unskl,0.229072,NaN,NaN,NaN,NaN
7198,sm_test,CO,min,skl,0.376623,NaN,NaN,NaN,NaN
7199,sm_test,CO,min,unskl,0.108725,NaN,NaN,NaN,NaN
7200,sm_test,CO,oil,skl,1.636361,NaN,NaN,NaN,NaN


In [129]:
3618*2

7236

In [127]:
ld0_breakouts

,r,s,incwage,incwage_total,hh_shr
0,AK,adm,37926585.0,4.015541e+09,0.009445
1,AK,agr,3816258.0,4.015541e+09,0.000950
2,AK,air,45855090.0,4.015541e+09,0.011419
3,AK,alt,505881.0,4.015541e+09,0.000126
4,AK,amb,163818178.0,4.015541e+09,0.040796
...,...,...,...,...,...
3613,WY,wht,57072515.0,3.063165e+09,0.018632
3614,WY,wpd,5061685.0,3.063165e+09,0.001652
3615,WY,wrh,1433059.0,3.063165e+09,0.000468
3616,WY,wst,12486591.0,3.063165e+09,0.004076


In [126]:
len(ld0_breakouts)

3618

In [122]:
len(test2)

11195506

In [124]:
test2['sample'].unique()

['2010-2014, ACS 5-year', '2011-2015, ACS 5-year', '2012-2016, ACS 5-year', '2013-2017, ACS 5-year']
Categories (4, object): ['2010-2014, ACS 5-year' < '2011-2015, ACS 5-year' < '2012-2016, ACS 5-year' < '2013-2017, ACS 5-year']

In [119]:
list(test)

['year',
 'multyear',
 'sample',
 'serial',
 'cbserial',
 'hhwt',
 'cluster',
 'statefip',
 'strata',
 'gq',
 'pernum',
 'perwt',
 'educ',
 'educd',
 'labforce',
 'indnaics',
 'inctot',
 'incwage',
 'pwstate2',
 'NAICS',
 'cps_code',
 'state_x',
 'state_residence',
 'state_y',
 'state_work',
 'educ_new',
 'hh']

In [112]:
len(le0_breakouts)*2

5960

In [115]:
len(gams_dta_le0)

6830

In [116]:
gams_dta_le0

,file,flx,int,scn,esubw_oth,sector,seniority,short,value
0,sm_test,IA,IA,hh1,skl,0.273961,NaN,NaN,NaN
1,sm_test,IA,IA,hh1,unskl,0.765274,NaN,NaN,NaN
2,sm_test,IA,IA,hh2,skl,1.827777,NaN,NaN,NaN
3,sm_test,IA,IA,hh2,unskl,2.620903,NaN,NaN,NaN
4,sm_test,IA,IA,hh3,skl,4.429970,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6825,sm_test,CO,CO,hh3,unskl,5.436943,NaN,NaN,NaN
6826,sm_test,CO,CO,hh4,skl,37.754607,NaN,NaN,NaN
6827,sm_test,CO,CO,hh4,unskl,8.374432,NaN,NaN,NaN
6828,sm_test,CO,CO,hh5,skl,58.633981,NaN,NaN,NaN


In [31]:
gams_rssk_m

,r,q,h,sk,incwage,s


In [29]:
gams_rqhs_lm_gr

,r,q,h,s,incwage_total


In [28]:
gams_rqhs_lm

,r,q,s,h,sk,incwage,census_region_r,census_region_q


In [27]:
gams_rqhs_lm1

,sk,incwage,census_region_r,census_region_q,r,q,h,s,incwage_total,skl_shr


In [26]:
merge_gams[merge_gams['_merge']=='both']

,incwage,census_region_r,census_region_q,s,incwage_total,skl_shr,r,q,h,sk,pop,_merge,census_region


#### Deal with missing combinations in CPS data for r,q,sk

In [22]:
##outer merge CPS data with gams table with indicator
merge_gams=gams_rqhs_lm1.merge(le00,on=['r','q','h','sk'],how='outer',indicator=True)
###get census region and assign average skilled and unskilled breakouts by census region for missing combinations
merge_gams['census_region']=np.where(merge_gams['r'].isin(midwest),'midwest','')
merge_gams['census_region']=np.where(merge_gams['r'].isin(south),'south',merge_gams['census_region'])
merge_gams['census_region']=np.where(merge_gams['r'].isin(west),'west',merge_gams['census_region'])
merge_gams['census_region']=np.where(merge_gams['r'].isin(northeast),'northeast',merge_gams['census_region'])

##get skilled unskilled breakouts by industry and census region

census_avg_tot0=merge_gams.groupby(['census_region','h'],as_index=False).agg({'incwage':sum})
census_avg_tot0=census_avg_tot0.rename(columns={'incwage':'incwage_tot'})
census_avg0=merge_gams.groupby(['census_region','h','sk'],as_index=False).agg({'incwage':sum})

census_avg0=census_avg0.merge(census_avg_tot0,on=['census_region','h'],how='inner')
census_avg0['census_share']=census_avg0['incwage']/census_avg0['incwage_tot']
del census_avg0['incwage']
del census_avg0['incwage_tot']

# merge census averages onto original merged dataframe
merge_gams_cs=merge_gams.merge(census_avg0,on=['census_region','h','sk'],how='inner')

##keep only inner values and values missing from CPS

final0=merge_gams_cs[merge_gams_cs['_merge'].isin(['right_only','both'])]

##get skilled unskilled breakouts by industry (ignore missing values for now)

final_tot0=final0.groupby(['r','q','h'],as_index=False).agg({'incwage':sum})
final_tot0=final_tot0.rename(columns={'incwage':'incwage_tot'})


final_shrs0=final0.merge(final_tot0,on=['r','q','h'],how='inner')
final_shrs0['skill_shr']=final_shrs0['incwage']/final_shrs0['incwage_tot']
del final_shrs0['incwage']
del final_shrs0['incwage_tot']

#create tag to define missing combinations in final dataset

final_shrs0['combo']=final_shrs0['r']+"_"+final_shrs0['q']+"_"+final_shrs0['h']+"_"+final_shrs0['sk']

## get list of missing combionations using _merge indicator var

missing_combos0=final_shrs0[final_shrs0['_merge']=='right_only']['r']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['q']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['h']+"_"+final_shrs0[final_shrs0['_merge']=='right_only']['sk']

# ###if region sector combination is not missing, use census region values, otherwise keep original share value
# final_shrs0['skill_shr']=np.where(final_shrs0['combo'].isin(missing_combos0),final_shrs0['census_share'],final_shrs0['skill_shr'])
# final_shrs0['skill_shr']=np.where(final_shrs0['combo']=='ME_NJ_hh1',final_shrs0['census_share'],final_shrs0['skill_shr'])




# ###limit dataframe

# final_shrs0=final_shrs0[['r','q','s','h','sk','skill_shr','combo','pop','census_share']]

# #chksum to see that all proportions add to 1
# check_sum0=final_shrs0.groupby(['r','q','s','h'],as_index=False).agg({'skill_shr':sum})
# check_sum0=check_sum0[check_sum0['skill_shr']!=1]


# #chksum to see that no zeros exist
# check_sum10=final_shrs0.groupby(['r','q','s','h','sk'],as_index=False).agg({'skill_shr':sum})
# check_sum10=check_sum10[check_sum10['skill_shr']==0]

# zeros=check_sum10['r']+"_"+check_sum10['q']+"_"+check_sum10['s']+"_"+check_sum10['h']


# #replace 0s with census values

# final_shrs0['skill_shr']=np.where(final_shrs0['combo'].isin(zeros),final_shrs0['census_share'],final_shrs0['skill_shr'])



# #check zeros agains
# check_sum10=final_shrs0.groupby(['r','q','s','h','sk'],as_index=False).agg({'skill_shr':sum})
# check_sum10=check_sum10[check_sum10['skill_shr']==0]

# final_shrs0_f=final_shrs0[['r','q','s','h','sk','skill_shr']]



In [23]:
missing_combos0

0         IA_IA_hh1_skl
1       IA_IA_hh1_unskl
2         IA_OH_hh1_skl
3       IA_OH_hh1_unskl
4         IA_WI_hh1_skl
             ...       
6825    CO_MT_hh5_unskl
6826      CO_WY_hh5_skl
6827    CO_WY_hh5_unskl
6828      CO_CO_hh5_skl
6829    CO_CO_hh5_unskl
Length: 6830, dtype: object

In [ ]:
final_shrs0[final_shrs0['census_share']==0]

In [ ]:
zeros

In [ ]:
check_sum10

In [ ]:
missing_combos0

In [ ]:
check_sum0

In [ ]:
final_shrs0[final_shrs0['combo'].isin(missing_combos0)]['pop'].sum()

In [ ]:
469.6733563736173/9215

In [ ]:
len(final_shrs0)

In [ ]:
missing_combos0